In [10]:
import pandas as pd
import numpy as np
from collections import deque

In [11]:
df = pd.read_table(filepath_or_buffer="soc-LiveJournal1.txt", skiprows=4, names=["FromNodeId", "ToNodeId"], index_col="FromNodeId")

df

,ToNodeId
FromNodeId,
0,1
0,2
0,3
0,4
0,5
...,...
4847570,476973
4847570,525852
4847570,537376


In [12]:
len(df.index.unique())

4308452

In [13]:
edges_dict = dict(df.groupby(df.index).apply(lambda x: np.array(x)))

for key in edges_dict.keys():
    edges_dict[key] = edges_dict[key].flatten()

In [14]:
edges_dict

{0: array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46]),
 1: array([      0,       1,       5,      20,     135,    2409,    8715,
           8932,   10623,   12347,   12846,   13840,   13845,   14005,
          20075,   21556,   22939,   23520,   28193,   29724,   29791,
          29826,   30691,   31232,   31435,   32317,   32489,   34394,
          35589,   35605,   35606,   35613,   35633,   35648,   35678,
          38737,   43447,   44846,   44887,   49226,   49985,   50466,
          50535,   50544,   50926,   51065,   51167,   51357,   52148,
          52904,   52914,   54313,   54627,   54683,   55132,   55655,
          55677,   57039,   58204,   58609,   58648,   58680,   58684,
          58719,   58943,   59027,   59949,   59959,   61387,   61398,
          61813,   65299,   70584,   70827,   71596,   72863,   72873,
   

In [16]:
# function to compare items of list to dict keys 
def compare(list1, dict1): 
    return [item for item in list1 if item not in dict1.keys()]

In [17]:
# numbers for which there are no nodes in the graph
compare(list1=[i for i in range(0, len(edges_dict.keys()))], dict1=edges_dict)

[152,
 260,
 264,
 268,
 272,
 296,
 333,
 334,
 342,
 347,
 360,
 365,
 395,
 412,
 431,
 435,
 511,
 530,
 650,
 700,
 707,
 720,
 777,
 780,
 782,
 785,
 794,
 796,
 847,
 921,
 969,
 1032,
 1045,
 1046,
 1079,
 1111,
 1214,
 1316,
 1382,
 1386,
 1400,
 1426,
 1588,
 1622,
 1631,
 1738,
 1780,
 1787,
 1792,
 1797,
 1895,
 1919,
 1934,
 1973,
 1978,
 1980,
 1981,
 2029,
 2074,
 2088,
 2091,
 2277,
 2338,
 2351,
 2371,
 2394,
 2409,
 2415,
 2437,
 2713,
 2763,
 2776,
 2879,
 2893,
 2914,
 3119,
 3127,
 3128,
 3151,
 3203,
 3208,
 3218,
 3220,
 3407,
 3456,
 3467,
 3554,
 3557,
 3580,
 3685,
 3704,
 3720,
 3838,
 4007,
 4039,
 4040,
 4080,
 4085,
 4133,
 4154,
 4155,
 4159,
 4195,
 4214,
 4382,
 4458,
 4479,
 4494,
 4504,
 4516,
 4546,
 4581,
 4638,
 4688,
 4749,
 4755,
 4775,
 4792,
 4809,
 4860,
 4863,
 4937,
 4944,
 4980,
 4982,
 5080,
 5084,
 5145,
 5163,
 5180,
 5183,
 5208,
 5213,
 5214,
 5216,
 5222,
 5233,
 5354,
 5357,
 5402,
 5429,
 5438,
 5445,
 5463,
 5465,
 5502,
 5514,
 5

MinHeap implementation for min distance retrieval from a pair (distance, node)

In [162]:
def heappop(pq):

    if len(pq) == 1:
        return pq.pop()
    else:
        SmallestValue = pq[0]
        pq[0] = pq.pop()
        HeapifyDown(pq, 0)
        return SmallestValue


def heappush(pq, pair):
    pq.append(pair)
    HeapifyUp(pq, len(pq) - 1)


def HeapifyDown(pq, index):
    SmallestValueIndex = index
    LeftChildIndex = 2 * index + 1
    RightChildIndex = 2 * index + 2
    if (LeftChildIndex < len(pq)) and (pq[LeftChildIndex][0] < (pq[SmallestValueIndex][0])):
        SmallestValueIndex = LeftChildIndex
    if (RightChildIndex < len(pq)) and (pq[RightChildIndex][0] < pq[SmallestValueIndex][0]):
        SmallestValueIndex = RightChildIndex
    if (SmallestValueIndex != index):
        pq[index], pq[SmallestValueIndex] = pq[SmallestValueIndex], pq[index]
        HeapifyDown(pq, SmallestValueIndex)


def HeapifyUp(pq, index):
    IndexParent = (index - 1) // 2
    if (IndexParent >= 0) and (pq[index][0] < pq[IndexParent][0]):
        pq[index], pq[IndexParent] = pq[IndexParent], pq[index]
        HeapifyUp(pq, IndexParent)


modified djikstra to keep track of the shortest distance, shortest path of each connected node 

In [163]:
def wajikstra(graph, start):

    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    pq = [(0, start)]
    shorted_path = {node: [] for node in graph}
    shorted_path[start] = [start]
    disconnected_nodes = []
    
    while pq:
        (dist, node) = heappop(pq)
        for neighbor in graph[node]:
            NewDistance = dist + 1
            try:
                if NewDistance < distances[neighbor]:
                    distances[neighbor] = NewDistance
                    heappush(pq, (NewDistance, neighbor))
                    # pq.append((new_dist, neighbor))  
                    # pq[neighbor] = new_dist 
                    shorted_path[neighbor] = shorted_path[node] + [neighbor]

            except KeyError:
                disconnected_nodes.append(neighbor)
                continue

    return distances, shorted_path, disconnected_nodes


In [184]:
distances = {} 
shortest_path = {}
disconnected_nodes = {}
roll_numbers = [1714, 2684, 1783]
for roll_number in roll_numbers:
    distances[roll_number], shortest_path[roll_number], disconnected_nodes = wajikstra(graph=edges_dict, start=2684)
    pd.DataFrame.from_dict(data=shortest_path, orient='index').to_csv(f'shorted_path_{roll_number}.csv',sep='\t', header=False)

In [185]:
shortest_path

{1714: {0: [2684, 131409, 52, 0],
  1: [2684, 21556, 1],
  2: [2684, 55280, 137399, 2],
  3: [2684, 465562, 151020, 3],
  4: [2684, 636019, 157604, 4],
  5: [2684, 128968, 55655, 5],
  6: [2684, 88226, 128117, 6],
  7: [2684, 50985, 61834, 7],
  8: [2684, 72983, 72990, 1487815, 8],
  9: [2684, 219665, 298563, 9],
  10: [2684, 2646, 51246, 10],
  11: [2684, 32631, 11],
  12: [2684, 51472, 12],
  13: [2684, 636019, 13],
  14: [2684, 133372, 185953, 574771, 14],
  15: [2684, 224641, 290775, 15],
  16: [2684, 197369, 339398, 16],
  17: [2684, 288871, 454654, 17],
  18: [2684, 197369, 147715, 18],
  19: [2684, 136240, 3926, 19],
  20: [2684, 50541, 453961, 20],
  21: [2684, 131409, 52, 21],
  22: [2684, 86955, 41439, 43400, 22],
  23: [2684, 219665, 499047, 136346, 23],
  24: [2684, 133372, 71409, 53, 24],
  25: [2684, 131409, 52, 0, 25],
  26: [2684, 224634, 197954, 26],
  27: [2684, 636019, 157604, 27],
  28: [2684, 77739, 146144, 28],
  29: [2684, 707468, 12647, 29],
  30: [2684, 133372,

In [186]:

for node in disconnected_nodes:
    try:
        if len(edges_dict[node]) > 0:
            print(node)
    except KeyError:
        continue

In [187]:
print('There are {} disconnected nodes in the provided data'.format(len(disconnected_nodes)))


There are 1997475 disconnected nodes in the provided data


In [188]:
if 21556 in edges_dict[2684]:
    if 1 in edges_dict[21556]:
        if 0 in edges_dict[1]:
            if 23 in edges_dict[0]:
                print("best")

best
